In [25]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras

In [4]:
# Check GPU availability and set GPU as default device
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Enable memory growth for GPUs
        print(f"Using GPU: {gpus}")
    except RuntimeError as e:
        print(f"Error initializing GPU: {e}")
else:
    print("No GPU detected, using CPU.")

No GPU detected, using CPU.


In [5]:
tf.config.list_physical_devices('GPU')

[]

In [6]:
# Load datasets
file_paths = [
    r"E:/Random Python Scripts/CICIDS/CICIDS2017/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",
    r"E:/Random Python Scripts/CICIDS/CICIDS2017/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
    r"E:/Random Python Scripts/CICIDS/CICIDS2017/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv",
    r"E:/Random Python Scripts/CICIDS/CICIDS2017/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv",
    r"E:/Random Python Scripts/CICIDS/CICIDS2017/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
    r"E:/Random Python Scripts/CICIDS/CICIDS2017/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
    r"E:/Random Python Scripts/CICIDS/CICIDS2017/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv",
    r"E:/Random Python Scripts/CICIDS/CICIDS2017/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv"
]

# Concatenate all datasets
df = pd.concat([pd.read_csv(file) for file in file_paths], ignore_index=True)

In [7]:
# Convert from multi-class to binary classification
attack_labels = df[' Label'].unique()
attack_labels = [label for label in attack_labels if label != 'BENIGN']
df[' Label'].replace(attack_labels, "Attack", inplace=True)

C:\Users\Siddharth\AppData\Local\Temp\ipykernel_7964\14994437.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[' Label'].replace(attack_labels, "Attack", inplace=True)


In [8]:
# Remove whitespace from column names
df.columns = [col.strip() for col in df.columns]

In [9]:
# Shuffle dataset
df = df.sample(frac=1, random_state=8).reset_index(drop=True)

In [10]:
# Encode labels
y = LabelEncoder().fit_transform(df['Label'])

In [11]:
# Prepare features
x = df.drop(columns=['Label'], axis=1).astype('float32')

In [12]:
# Handle missing and infinite values
x.replace([np.inf, -np.inf], np.nan, inplace=True)
x.fillna(x.mean(), inplace=True)
x[x < 0] = np.nan
x.fillna(x.min(), inplace=True)

In [13]:
# Scale features
scaler = StandardScaler()
x = pd.DataFrame(scaler.fit_transform(x), index=x.index, columns=x.columns)

In [14]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=8, stratify=y)

In [19]:
# Build model function
def build_model():
    # Creating layers
    inputs = keras.layers.Input(shape=(X_train.shape[1],))
    
    x = keras.layers.Dense(units=78, activation='relu')(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.1)(x)
    
    x = keras.layers.Dense(units=15, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)

    x = keras.layers.Dense(units=7, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Dense(units=15, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.3)(x)
    
    x = keras.layers.Dense(units=35, activation='relu')(x)
    outputs = keras.layers.Dense(units=2, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer=keras.optimizers.Adam(learning_rate=0.001), 
                  metrics=['accuracy'])
    
    return model

In [16]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5)

In [20]:
model = build_model()

In [21]:
# Display model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 78)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 78)                  │           6,162 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 78)                  │             312 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 78)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 15)                  │           1,185 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 15)                  │              60 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 7)                   │             112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 7)                   │              28 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 15)                  │             120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 15)                  │              60 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 15)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 35)                  │             560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │              72 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,671 (33.87 KB)

 Trainable params: 8,441 (32.97 KB)

 Non-trainable params: 230 (920.00 B)

In [23]:
# Train model
model.fit(X_train, 
          y_train, 
          epochs=100, 
          batch_size=128, 
          validation_split=0.2, 
          verbose=1, 
          callbacks=[early_stop]
         )

Epoch 1/100
12385/12385 ━━━━━━━━━━━━━━━━━━━━ 53s 4ms/step - accuracy: 0.9568 - loss: 0.1017 - val_accuracy: 0.9745 - val_loss: 0.0500
Epoch 2/100
12385/12385 ━━━━━━━━━━━━━━━━━━━━ 43s 4ms/step - accuracy: 0.9749 - loss: 0.0549 - val_accuracy: 0.9791 - val_loss: 0.0469
Epoch 3/100
12385/12385 ━━━━━━━━━━━━━━━━━━━━ 83s 4ms/step - accuracy: 0.9787 - loss: 0.0478 - val_accuracy: 0.9769 - val_loss: 0.0481
Epoch 4/100
12385/12385 ━━━━━━━━━━━━━━━━━━━━ 44s 4ms/step - accuracy: 0.9831 - loss: 0.0388 - val_accuracy: 0.9913 - val_loss: 0.0275
Epoch 5/100
12385/12385 ━━━━━━━━━━━━━━━━━━━━ 49s 4ms/step - accuracy: 0.9859 - loss: 0.0339 - val_accuracy: 0.9895 - val_loss: 0.0254
Epoch 6/100
12385/12385 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.9866 - loss: 0.0318 - val_accuracy: 0.9889 - val_loss: 0.0247
Epoch 7/100
12385/12385 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.9868 - loss: 0.0317 - val_accuracy: 0.9892 - val_loss: 0.0231
Epoch 8/100
12385/12385 ━━━━━━━━━━━━━━━━━━━━ 44s 4ms/step - ac

In [26]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_classes))

Test Loss: 0.025916405022144318
Test Accuracy: 0.9911613464355469
26539/26539 ━━━━━━━━━━━━━━━━━━━━ 53s 2ms/step
              precision    recall  f1-score   support

           0       0.96      0.99      0.98    167294
           1       1.00      0.99      0.99    681929

    accuracy                           0.99    849223
   macro avg       0.98      0.99      0.99    849223
weighted avg       0.99      0.99      0.99    849223



In [27]:
# Save the model
model.save("DNN_BC.h5")
print("Model saved successfully.")

Model saved successfully.
